# Baza BASiW
Przypadki i zgony w podziale na:
* Datę
* Województwo
* Powiat
* Wiek
* Płeć
* Stan zaszczepienia
* Choroby współistniejące
* Upośledzenie odporności
* Więcej...

## ŚCIĄGNIJ AKTUALNE DANE RĘCZNIE!!!
https://basiw.mz.gov.pl/index.html#/visualization?id=3761

W formatowaniu danych doklejam kolumnę z nazwą województwa, bo jest to dla mnie wygodniejsze do czytania, gdy wyświetlam tabelę dla kontroli.

In [1]:
"""
Set PYTHONPATH for modules in parallel directory

https://stackoverflow.com/questions/3108285/in-python-script-how-do-i-set-pythonpath/3108301

"""

import sys

try:
    sys.path.index('..') # Or os.getcwd() for this directory
except ValueError:
    sys.path.append('..') # Or os.getcwd() for this directory

#--------------------------------------------------------------
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import datetime
from share.helper_functions import was_modified_today, display_all
# from share.basiw_functions import BASiWformat

In [2]:

class RawDf:
    df : pd.DataFrame
    def __init__(self, df : pd.DataFrame) -> None:
        self.df=df

class BASiWformat:
    image_dir : str
    data_dir : str
    data_file_deaths : str
    data_file_cases : str
    teryt_file : str
    path_deaths : str
    path_cases : str
    path_teryt : str
    _dfd_raw : RawDf
    _dfc_raw : RawDf
    _df_teryt_raw : RawDf
    df_woj : pd.DataFrame
    teryt_dict : dict
    dfd : pd.DataFrame

    def __init__(self,
        image_dir : str,
        data_dir : str,
        data_file_deaths : str,
        data_file_cases : str,
        teryt_file : str
        ) -> None:

        self.image_dir=image_dir
        self.data_dir=data_dir
        self.data_file_deaths=data_file_deaths
        self.data_file_cases=data_file_cases
        self.teryt_file=teryt_file
        self.path_deaths = os.sep.join([self.data_dir,self.data_file_deaths])
        self.path_cases = os.sep.join([self.data_dir,self.data_file_cases])
        self.path_teryt = os.sep.join([self.data_dir,self.teryt_file])
        self.read_data()
        self.make_teryt_dict()
        self.dfd = self.format_df('data_rap_zgonu', self._dfd_raw)
        return


    def read_data(self) -> None:
        # if was_modified_today(path_deaths):
        #     print('Data up to date')
        #     dfd = pd.read_csv(path_deaths, encoding = 'cp1250', sep = ';')
        #     dfc = pd.read_csv(path_cases, encoding = 'cp1250', sep = ';')
        # else:
        #     print('Old data, need to download new data!')
        self._dfd_raw = RawDf(pd.read_csv(self.path_deaths, encoding = 'cp1250', sep = ';',low_memory=False))
        self._dfc_raw= RawDf(pd.read_csv(self.path_cases, encoding = 'cp1250', sep = ';', low_memory=False))
        self._df_teryt_raw= RawDf(pd.read_csv(self.path_teryt, sep = ';'))
        return
        
    def make_teryt_dict(self) -> None:
        self.df_woj = self._df_teryt_raw.df.query('NAZWA_DOD == "województwo"')[['WOJ', 'NAZWA']]
        self.df_woj['NAZWA'] = self.df_woj['NAZWA'].str.title()
        self.df_woj.index = self.df_woj.pop('WOJ')
        self.teryt_dict = self.df_woj.to_dict()['NAZWA']
        return

    def format_df(self, col : str, df_raw : RawDf) -> pd.DataFrame:
        df = df_raw.df.copy(deep=True)
        df[col] = pd.to_datetime(df[col], format = "%Y-%m-%d")
        df['Województwo'] = df['teryt_woj']
        df['Województwo'].replace(self.teryt_dict, inplace=True)
        return df

    def dfd_from(self, start_date : datetime.date) -> pd.DataFrame:
        return self.dfd[self.dfd['data_rap_zgonu'] >= pd.to_datetime(start_date)]

In [3]:
basiw = BASiWformat(
    image_dir = './images',
    data_dir = './basiw_data',
    data_file_deaths = 'zgony.csv',
    data_file_cases = 'zakazenia.csv',
    teryt_file = 'TERC_Urzedowy_2021-12-14.csv'
    )

basiw.dfd


,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
0,2021-01-02,4,407,K,67.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
1,2021-01-02,4,407,K,90.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
2,2021-01-02,4,407,M,69.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
3,2021-01-02,4,415,M,72.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
4,2021-01-02,4,415,M,77.0,75-84,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


In [4]:
wave4_date = datetime.date(2021, 7, 14)
dfd4 = basiw.dfd_from(wave4_date)
dfd4


,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
45248,2021-07-14,2,201,K,95.0,95+,N,NaN,NaN,N,1,N,Dolnośląskie
45249,2021-07-14,4,410,K,89.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
45250,2021-07-14,14,1412,K,45.0,45-54,N,NaN,NaN,N,1,N,Mazowieckie
45251,2021-07-14,16,1610,M,65.0,65-74,T,Astra Zeneca,jedna_dawka,N,1,N,Opolskie
45252,2021-07-14,18,1808,M,74.0,65-74,T,NaN,NaN,N,1,N,Podkarpackie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


### Liczba zgonów na COVID-19 w województwach na 100 tys. mieszkańców<br> w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br> (Od 14.07.2021)

**Potrzebny jest słownik z danymi: Populacja Polski / regionu wg wojwództw !!!**
/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/GUS/gus_data/wojewodztwa_ludnosc_plec_30.06.2021_GUS.csv

In [5]:
def get_popwoj_df() -> pd.DataFrame:
    dfpopwoj = pd.read_csv('/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/GUS/gus_data/wojewodztwa_ludnosc_plec_30.06.2021_GUS.csv')
    dfpopwoj['Województwo'] = dfpopwoj['Województwo'].str.title()
    dfpopwoj.drop(['TERYT'], axis='columns', inplace=True)
    dfpopwoj.index = dfpopwoj['Województwo']
    dfpopwoj.drop(['Województwo'], axis='columns' , inplace = True)
    return dfpopwoj

dfpopwoj  = get_popwoj_df()
dfpopwoj = dfpopwoj[['Ludność']]

# dfpopwoj

# dfpopwoj.to_dict()['Ludność']

In [6]:
 def prepare_df(deaths_df: pd.DataFrame, from_date : datetime.date) -> pd.DataFrame:
    base_df = deaths_df[deaths_df['data_rap_zgonu'] >= pd.to_datetime(from_date)]

    df = base_df.groupby('Województwo').sum()
    df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Województwo'] = df.index
    df['Ludność'] = df['Województwo'].replace(dfpopwoj.to_dict()['Ludność'])
    df

    Q = f'(w_pelni_zaszczepiony == "N")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Zgony nie w pełni zaszczepionych'] = df1

    Q = f'(w_pelni_zaszczepiony == "T")'

    df1 = base_df.query(Q).groupby('Województwo').sum()
    df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
    df['Zgony w pełni zaszczepionych'] = df1




    df['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony nie w pełni zaszczepionych'] / df['Ludność'] * 1e5
    df['Zgony w pełni zaszczepionych na 100 tys. mieszkańców'] = df['Zgony w pełni zaszczepionych'] / df['Ludność'] * 1e5

    df = df.sort_values(['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'], ascending=False)
    return df

    
def plot_df(from_date : datetime.date, df : pd.DataFrame, image_dir : str) -> pd.DataFrame:
    nie = 'Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'
    tak = 'Zgony w pełni zaszczepionych na 100 tys. mieszkańców'

    fig = go.Figure(
        data=[
            go.Bar(
                name = nie, 
                x=df['Województwo'], 
                y=df[nie], 
                offsetgroup=1,
                text = df[nie]
                ),
            go.Bar(
                name = tak, 
                x=df['Województwo'], 
                y=df[tak], 
                offsetgroup=2,
                text = df[tak]
                )
        ]
    )

    fig.update_traces(
        texttemplate='%{text:2.1f}', 
        textposition='outside',
        )

    fig.update_layout(
        barmode='group', 
        width=800, 
        height=500,
        title = "Liczba zgonów na COVID-19 w województwach na 100 tys. mieszkańców<br>\
        w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(Od " + str(from_date) + ")",
        yaxis_title="Liczba zgonów na 100 tys.",
        legend_title="",
        uniformtext_minsize=6, 
        uniformtext_mode='show',
        legend=dict(
            y=-0.5,
            xanchor="center",
            x=0.5
            ),
        title_xanchor='center',
        title_x = 0.5,
        title_font_size=16
        )
    fig.update_xaxes(tickangle=33)
    fig.show()
    fig.write_image(os.sep.join([image_dir, 'zgony_szczep_1e5_woj_od'+str(from_date)+'.jpg']), scale=4)
    return


In [7]:
from_date = datetime.date(2021,7,14)



df = prepare_df(basiw.dfd, from_date)
plot_df(from_date, df, basiw.image_dir)

### Liczba zgonów na COVID-19 w Polsce na 100 tys. mieszkańców<br> w podziale na grupy wiekowe<br> (Od 14.07.2021)

**Potrzebny jest słownik z danymi: Populacja Polski / regionu wg grup wiekowych !!!**
/home/ochab/koronawirus_PAN/gov.pl/bitbucket/govpl/Eurostat/eurostat_data/Poland_2020_population.csv

In [8]:
pop_file = "../Eurostat/eurostat_data/Poland_2020_population.csv"
pop_df = pd.read_csv(pop_file , index_col=0)
# print(pop_df['Population 2020 F'].sum() + pop_df['Population 2020 M'].sum())
# pop_df['Population 2020 T'].sum()

pop_df.rename(
    columns = {
        'Population 2020 T' : 'mieszkańców w danej grupie wiekowej',
        'Population 2020 F' : 'kobiet w danej grupie wiekowej',
        'Population 2020 M' : 'mężczyzn w danej grupie wiekowej'
        }, inplace = True
    )
pop_df

,mieszkańców w danej grupie wiekowej,kobiet w danej grupie wiekowej,mężczyzn w danej grupie wiekowej
Age group,,,
0-18,6909092.0,3362871.0,3546221.0
19-24,2391135.0,1169959.0,1221176.0
25-34,5378329.0,2637687.0,2740642.0
35-44,6101624.0,3008948.0,3092676.0
45-54,4789676.0,2395882.0,2393794.0
55-64,5104262.0,2668881.0,2435381.0
65-74,4195973.0,2362488.0,1833485.0
75-84,1912001.0,1218241.0,693760.0
85-94,756428.0,540453.0,215975.0


In [9]:
pop_woj_df = dfpopwoj
pop_woj_df.rename(
    columns = {
        'Ludność' : 'mieszkańców w województwie',
        'Kobiety' : 'kobiet w województwie',
        'Mężczyźni' : 'mężczyzn w województwie'
        }, inplace = True
    )
pop_woj_df

,mieszkańców w województwie
Województwo,
Dolnośląskie,2884011
Kujawsko-Pomorskie,2054163
Lubelskie,2086403
Lubuskie,1003150
Łódzkie,2426806
Małopolskie,3408505
Mazowieckie,5419963
Opolskie,973298
Podkarpackie,2115578


In [10]:
def group_by(base_df: pd.DataFrame, x_col: str, x_col_name : str, denominator_df: pd.DataFrame, denominator_col : str) -> pd.DataFrame:
   df = base_df.groupby(x_col).sum()
   df.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
   df = pd.concat([df,denominator_df[denominator_col]], axis='columns')
   # display(df)

   Q = f'(w_pelni_zaszczepiony == "N")'

   df1 = base_df.query(Q).groupby(x_col).sum()
   df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
   df['Zgony nie w pełni zaszczepionych '  + denominator_col] = df1
   
   
   Q = f'(w_pelni_zaszczepiony == "T")'

   df1 = base_df.query(Q).groupby(x_col).sum()
   df1.drop(['wiek', 'teryt_pow', 'teryt_woj'], axis = 'columns' , inplace=True)
   df['Zgony w pełni zaszczepionych '  + denominator_col] = df1
   df[x_col]=df.index
   df.rename({x_col : x_col_name}, axis=1, inplace=True)
   return df
 
 
def prepare_df2(deaths_df: pd.DataFrame, x_col: str, x_col_name : str, denominator_df: pd.DataFrame, denominator_col : str, from_date : datetime.date) -> pd.DataFrame:
   base_df = deaths_df[deaths_df['data_rap_zgonu'] >= pd.to_datetime(from_date)]

   df = group_by(base_df, x_col, x_col_name, denominator_df, denominator_col)
   # display(df)

   df['Zgony nie w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col] =\
      df['Zgony nie w pełni zaszczepionych '  + denominator_col].to_numpy() / df[denominator_col].to_numpy() * 1e5
   # display(df)
   
   df['Zgony w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col] =\
      df['Zgony w pełni zaszczepionych '  + denominator_col].to_numpy() / df[denominator_col].to_numpy() * 1e5

   # df = df.sort_values(['Zgony nie w pełni zaszczepionych na 100 tys. mieszkańców'], ascending=False)
   return df
   

In [11]:
from_date = datetime.date(2021,7,14)

df = prepare_df2(
    deaths_df = basiw.dfd, 
    x_col ='kat_wiek', 
    x_col_name = 'Grupa wiekowa',
    denominator_df=pop_df,  
    denominator_col='mieszkańców w danej grupie wiekowej', 
    from_date=from_date
    )

df

,liczba_zaraportowanych_zgonow,mieszkańców w danej grupie wiekowej,Zgony nie w pełni zaszczepionych mieszkańców w danej grupie wiekowej,Zgony w pełni zaszczepionych mieszkańców w danej grupie wiekowej,Grupa wiekowa,Zgony nie w pełni zaszczepionych mieszkańców w danej grupie wiekowej na 100 tys. mieszkańców w danej grupie wiekowej,Zgony w pełni zaszczepionych mieszkańców w danej grupie wiekowej na 100 tys. mieszkańców w danej grupie wiekowej
0-18,9,6909092.0,8,1.0,0-18,0.115789,0.014474
19-24,18,2391135.0,17,1.0,19-24,0.710959,0.041821
25-34,99,5378329.0,92,7.0,25-34,1.710568,0.130152
35-44,253,6101624.0,228,25.0,35-44,3.736710,0.409727
45-54,622,4789676.0,542,80.0,45-54,11.316006,1.670259
55-64,1465,5104262.0,1172,293.0,55-64,22.961204,5.740301
65-74,3565,4195973.0,2432,1133.0,65-74,57.960335,27.002080
75-84,3929,1912001.0,2714,1215.0,75-84,141.945532,63.545992
85-94,3536,756428.0,2622,914.0,85-94,346.629157,120.831064
95+,367,52344.0,299,68.0,95+,571.221152,129.909827


In [12]:
def plot_df2(
    x_col : str, 
    denominator_col : str, 
    title : str, 
    filename_str : str,  
    from_date : datetime.date, 
    df : pd.DataFrame, 
    image_dir : str,
    width=800, 
    height=500,
    legend_y=-0.25
    ) -> pd.DataFrame:

    nie = 'Zgony nie w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col
    tak = 'Zgony w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col

    fig = go.Figure(
        data=[
            go.Bar(
                name = nie, 
                x=df[x_col], 
                y=df[nie], 
                offsetgroup=1,
                text = df[nie]
                ),
            go.Bar(
                name = tak, 
                x=df[x_col], 
                y=df[tak], 
                offsetgroup=2,
                text = df[tak]
                )
        ]
    )

    fig.update_traces(
        texttemplate='%{text:2.1f}', 
        textposition='outside',
        )

    fig.update_layout(
        barmode='group', 
        width=width, 
        height=height,
        title = title , #"Liczba zgonów " + denominator_col + " na COVID-19 " + title_str + " na 100 tys. "+ denominator_col +
        # "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych<br>(od " + str(from_date) + ")",
        yaxis_title="Liczba zgonów na 100 tys.",
        legend_title="",
        uniformtext_minsize=6, 
        uniformtext_mode='show',
        legend=dict(
            y=legend_y,
            xanchor="center",
            x=0.5
            ),
        title_xanchor='center',
        title_x = 0.5,
        title_font_size=16
        )
    fig.update_xaxes(tickangle=33)
    fig.show()
    fig.write_image(os.sep.join([image_dir, 'zgony_szczep_1e5_'+ filename_str + '_od'+str(from_date)+'.jpg']), scale=4)
    return

In [13]:
x_col = "Grupa wiekowa"
title_str = "w Polsce" #"w województwach"
filename_str = "wiek_ogolem"
denominator_col = "mieszkańców w danej grupie wiekowej"
title = "Liczba zgonów na COVID-19 " + denominator_col + " " + title_str + "<br>na 100 tys. "+ denominator_col +\
        "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych (od " + str(from_date) + ")"
df_to_plot = df[:-1]

plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title=title , 
    filename_str=filename_str,  
    from_date = from_date, 
    df = df_to_plot, 
    image_dir = basiw.image_dir
    )

### Liczba zgonów na COVID-19 w Polsce, według płci, na 100 tys. mieszkańców<br> w podziale na grupy wiekowe<br> (Od 14.07.2021) 

In [14]:
basiw.dfd.query('plec=="M"')

,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
2,2021-01-02,4,407,M,69.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
3,2021-01-02,4,415,M,72.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
4,2021-01-02,4,415,M,77.0,75-84,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
5,2021-01-02,4,418,M,76.0,75-84,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
7,2021-01-02,4,464,M,67.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


In [15]:
from_date = datetime.date(2021,7,14)

dfM = prepare_df2(
    deaths_df = basiw.dfd.query('plec=="M"'), 
    x_col ='kat_wiek', 
    x_col_name = 'Grupa wiekowa',  
    denominator_df=pop_df,
    denominator_col='mężczyzn w danej grupie wiekowej', 
    from_date=from_date
    )

dfM = dfM[:-1].fillna(0)
dfM

,liczba_zaraportowanych_zgonow,mężczyzn w danej grupie wiekowej,Zgony nie w pełni zaszczepionych mężczyzn w danej grupie wiekowej,Zgony w pełni zaszczepionych mężczyzn w danej grupie wiekowej,Grupa wiekowa,Zgony nie w pełni zaszczepionych mężczyzn w danej grupie wiekowej na 100 tys. mężczyzn w danej grupie wiekowej,Zgony w pełni zaszczepionych mężczyzn w danej grupie wiekowej na 100 tys. mężczyzn w danej grupie wiekowej
0-18,7,3546221.0,7,0.0,0-18,0.197393,0.000000
19-24,10,1221176.0,10,0.0,19-24,0.818883,0.000000
25-34,71,2740642.0,66,5.0,25-34,2.408195,0.182439
35-44,163,3092676.0,147,16.0,35-44,4.753165,0.517351
45-54,427,2393794.0,370,57.0,45-54,15.456635,2.381157
55-64,862,2435381.0,664,198.0,55-64,27.264728,8.130145
65-74,2032,1833485.0,1292,740.0,65-74,70.466898,40.360297
75-84,1898,693760.0,1176,722.0,75-84,169.511070,104.070572
85-94,1355,215975.0,878,477.0,85-94,406.528533,220.858896
95+,103,11563.0,80,23.0,95+,691.861974,198.910317


In [16]:
x_col = "Grupa wiekowa"
title_str = "w Polsce" #"w województwach"
filename_str = "wiek_M"
denominator_col = "mężczyzn w danej grupie wiekowej"
title = "Liczba zgonów na COVID-19 " + denominator_col + " " + title_str + "<br>na 100 tys. "+ denominator_col +\
        "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych (od " + str(from_date) + ")"
df_to_plot = dfM


plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title=title, 
    filename_str=filename_str,  
    from_date = from_date, 
    df = df_to_plot, 
    image_dir = basiw.image_dir
    )

In [17]:
from_date = datetime.date(2021,7,14)

dfK = prepare_df2(
    deaths_df = basiw.dfd.query('plec=="K"'), 
    x_col ='kat_wiek', 
    x_col_name = 'Grupa wiekowa',
    denominator_df=pop_df,
    denominator_col='kobiet w danej grupie wiekowej', 
    from_date=from_date
    )

dfK = dfK[:-1].fillna(0)
dfK

,liczba_zaraportowanych_zgonow,kobiet w danej grupie wiekowej,Zgony nie w pełni zaszczepionych kobiet w danej grupie wiekowej,Zgony w pełni zaszczepionych kobiet w danej grupie wiekowej,Grupa wiekowa,Zgony nie w pełni zaszczepionych kobiet w danej grupie wiekowej na 100 tys. kobiet w danej grupie wiekowej,Zgony w pełni zaszczepionych kobiet w danej grupie wiekowej na 100 tys. kobiet w danej grupie wiekowej
0-18,2,3362871.0,1,1.0,0-18,0.029736,0.029736
19-24,8,1169959.0,7,1.0,19-24,0.598312,0.085473
25-34,28,2637687.0,26,2.0,25-34,0.985712,0.075824
35-44,85,3008948.0,76,9.0,35-44,2.525800,0.299108
45-54,192,2395882.0,169,23.0,45-54,7.053770,0.959980
55-64,598,2668881.0,503,95.0,55-64,18.846850,3.559544
65-74,1526,2362488.0,1133,393.0,65-74,47.957916,16.635005
75-84,2022,1218241.0,1529,493.0,75-84,125.508828,40.468183
85-94,2176,540453.0,1739,437.0,85-94,321.767110,80.858095
95+,264,40781.0,219,45.0,95+,537.014786,110.345504


In [18]:
x_col = "Grupa wiekowa"
title_str = "w Polsce" #"w województwach"
filename_str = "wiek_K"
denominator_col = "kobiet w danej grupie wiekowej"
df_to_plot = dfK
title = "Liczba zgonów na COVID-19 " + denominator_col + " " + title_str + "<br>na 100 tys. "+ denominator_col +\
        "<br>w podziale na w pełni zaszczepione i nie w pełni zaszczepione (od " + str(from_date) + ")"
plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title=title, 
    filename_str=filename_str,  
    from_date = from_date, 
    df = df_to_plot, 
    image_dir = basiw.image_dir
    )

### Zgony zaszczepionych i nie zaszczepionych w grupach wiekowych, z występowaniem chorób współistniejących i bez, na 100 tys. mieszkańców Polski w danej grupie wiekowej.

In [19]:
basiw.dfd.query('czy_wspolistniejace == "T"')

,data_rap_zgonu,teryt_woj,teryt_pow,plec,wiek,kat_wiek,czy_wspolistniejace,producent,dawka_ost,obniz_odpornosc,liczba_zaraportowanych_zgonow,w_pelni_zaszczepiony,Województwo
0,2021-01-02,4,407,K,67.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
1,2021-01-02,4,407,K,90.0,85-94,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
2,2021-01-02,4,407,M,69.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
3,2021-01-02,4,415,M,72.0,65-74,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
4,2021-01-02,4,415,M,77.0,75-84,T,NaN,NaN,N,1,N,Kujawsko-Pomorskie
...,...,...,...,...,...,...,...,...,...,...,...,...,...
58975,2021-12-13,32,3261,M,83.0,75-84,T,NaN,NaN,N,1,N,Zachodniopomorskie
58976,2021-12-13,32,3261,M,85.0,85-94,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58977,2021-12-13,32,3262,M,73.0,65-74,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie
58978,2021-12-13,32,3262,M,78.0,75-84,T,Pfizer,pelna_dawka,N,1,T,Zachodniopomorskie


In [20]:
from_date = datetime.date(2021,7,14)

df_wspol = prepare_df2(
    deaths_df = basiw.dfd.query('czy_wspolistniejace == "T"'), 
    x_col ='kat_wiek', 
    x_col_name = 'Grupa wiekowa',
    denominator_df=pop_df,
    denominator_col='mieszkańców w danej grupie wiekowej', 
    from_date=from_date
    )

df_wspol[:-1].fillna(0)

,liczba_zaraportowanych_zgonow,mieszkańców w danej grupie wiekowej,Zgony nie w pełni zaszczepionych mieszkańców w danej grupie wiekowej,Zgony w pełni zaszczepionych mieszkańców w danej grupie wiekowej,Grupa wiekowa,Zgony nie w pełni zaszczepionych mieszkańców w danej grupie wiekowej na 100 tys. mieszkańców w danej grupie wiekowej,Zgony w pełni zaszczepionych mieszkańców w danej grupie wiekowej na 100 tys. mieszkańców w danej grupie wiekowej
0-18,3,6909092.0,3,0.0,0-18,0.043421,0.000000
19-24,13,2391135.0,12,1.0,19-24,0.501854,0.041821
25-34,62,5378329.0,59,3.0,25-34,1.096995,0.055779
35-44,142,6101624.0,125,17.0,35-44,2.048635,0.278614
45-54,406,4789676.0,350,56.0,45-54,7.307384,1.169181
55-64,1014,5104262.0,788,226.0,55-64,15.438079,4.427672
65-74,2568,4195973.0,1730,838.0,65-74,41.230008,19.971530
75-84,2959,1912001.0,2014,945.0,75-84,105.334673,49.424660
85-94,2630,756428.0,1956,674.0,85-94,258.583765,89.102995
95+,252,52344.0,202,50.0,95+,385.908605,95.521932


In [21]:
x_col = "Grupa wiekowa"
title_str = "z chorobami współistniejącymi" #"w Polsce" #"w województwach"
filename_str = "wiek_ogolem_wspol"
denominator_col = "mieszkańców w danej grupie wiekowej"
title = "Liczba zgonów na COVID-19 " + denominator_col + " " + title_str + "<br>na 100 tys. "+ denominator_col +\
        "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych (od " + str(from_date) + ")"
df_to_plot = df_wspol[:-1].fillna(0)

plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title=title , 
    filename_str=filename_str,  
    from_date = from_date, 
    df = df_to_plot, 
    image_dir = basiw.image_dir
    )

In [22]:
from_date = datetime.date(2021,7,14)

df_bez_wspol = prepare_df2(
    deaths_df = basiw.dfd.query('czy_wspolistniejace == "N"'), 
    x_col ='kat_wiek', 
    x_col_name = 'Grupa wiekowa',
    denominator_df=pop_df, 
    denominator_col='mieszkańców w danej grupie wiekowej', 
    from_date=from_date
    )

df_bez_wspol[:-1].fillna(0)

,liczba_zaraportowanych_zgonow,mieszkańców w danej grupie wiekowej,Zgony nie w pełni zaszczepionych mieszkańców w danej grupie wiekowej,Zgony w pełni zaszczepionych mieszkańców w danej grupie wiekowej,Grupa wiekowa,Zgony nie w pełni zaszczepionych mieszkańców w danej grupie wiekowej na 100 tys. mieszkańców w danej grupie wiekowej,Zgony w pełni zaszczepionych mieszkańców w danej grupie wiekowej na 100 tys. mieszkańców w danej grupie wiekowej
0-18,6,6909092.0,5,1.0,0-18,0.072368,0.014474
19-24,5,2391135.0,5,0.0,19-24,0.209106,0.000000
25-34,37,5378329.0,33,4.0,25-34,0.613573,0.074373
35-44,111,6101624.0,103,8.0,35-44,1.688075,0.131113
45-54,216,4789676.0,192,24.0,45-54,4.008622,0.501078
55-64,451,5104262.0,384,67.0,55-64,7.523125,1.312629
65-74,997,4195973.0,702,295.0,65-74,16.730327,7.030550
75-84,970,1912001.0,700,270.0,75-84,36.610860,14.121332
85-94,906,756428.0,666,240.0,85-94,88.045392,31.728069


In [23]:
x_col = "Grupa wiekowa"
title_str = "bez chorób współistniejących" #"w Polsce" #"w województwach"
filename_str = "wiek_ogolem_bez_wspol"
denominator_col = "mieszkańców w danej grupie wiekowej"
title = "Liczba zgonów na COVID-19 " + denominator_col + " " + title_str + "<br>na 100 tys. "+ denominator_col +\
        "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych (od " + str(from_date) + ")"
df_to_plot = df_bez_wspol[:-1].fillna(0)

plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title=title , 
    filename_str=filename_str,  
    from_date = from_date, 
    df = df_to_plot, 
    image_dir = basiw.image_dir
    )

## Udział współistniejących wg województw

In [24]:
def plot_df3(df : pd.DataFrame, image_dir : str, filename : str) -> None:
    tak = 'Współistniejące [%]'
    nie = 'Brak współistniejących [%]'

    fig = go.Figure(
        data=[
            go.Bar(
                name = tak, 
                x=df['Województwo'], 
                y=df[tak], 
                offsetgroup=1,
                text = df[tak]
                ),
            go.Bar(
                name = nie, 
                x=df['Województwo'], 
                y=df[nie], 
                offsetgroup=2,
                text = df[nie]
                )
        ]
    )

    fig.update_traces(
        texttemplate='%{text:2.1f}', 
        textposition='outside',
        )

    fig.update_layout(
        barmode='group', 
        width=800, 
        height=500,
        title = "Procent zgonów na COVID-19 w województwach w podziale na występowanie chorób współistniejących<br>(Od 14.07.2021)",
        yaxis_title="Procent zgonów",
        legend_title="",
        uniformtext_minsize=8, 
        uniformtext_mode='show',
        legend=dict(
            y=-0.5,
            xanchor="center",
            x=0.5
            ),
        title_xanchor='center',
        title_x = 0.5,
        title_font_size=16
        )
    fig.update_xaxes(tickangle=33)
    fig.show()
    fig.write_image(os.sep.join([image_dir, filename]), scale=4) #zgony_wspolist_4fala.jpg
    return




In [25]:
wave4_date = datetime.date(2021, 7, 14)
dfd4 = basiw.dfd_from(wave4_date)

df = dfd4.groupby('Województwo').sum().drop(columns=['teryt_woj', 'teryt_pow', 'wiek'])

Q = f'(czy_wspolistniejace == "T")'
dfwt = dfd4.query(Q)

df['Współistniejące'] = dfwt.groupby('Województwo').sum().drop(columns=['teryt_woj', 'teryt_pow', 'wiek'])


Q = f'(czy_wspolistniejace == "N")'
dfwt = dfd4.query(Q)
df['Brak współistniejących'] = dfwt.groupby('Województwo').sum().drop(columns=['teryt_woj', 'teryt_pow', 'wiek'])
df['Współistniejące [%]'] = df['Współistniejące'] / df['liczba_zaraportowanych_zgonow'] *100
df['Brak współistniejących [%]'] = df['Brak współistniejących'] / df['liczba_zaraportowanych_zgonow'] *100
df = df.sort_values(['Współistniejące [%]'], ascending=False)
df['Województwo'] = df.index

df

,liczba_zaraportowanych_zgonow,Współistniejące,Brak współistniejących,Współistniejące [%],Brak współistniejących [%],Województwo
Województwo,,,,,,
Podlaskie,872,809,63,92.775229,7.224771,Podlaskie
Zachodniopomorskie,520,476,44,91.538462,8.461538,Zachodniopomorskie
Podkarpackie,943,832,111,88.229056,11.770944,Podkarpackie
Opolskie,435,378,57,86.896552,13.103448,Opolskie
Pomorskie,515,445,70,86.407767,13.592233,Pomorskie
Świętokrzyskie,461,392,69,85.032538,14.967462,Świętokrzyskie
Kujawsko-Pomorskie,767,608,159,79.269883,20.730117,Kujawsko-Pomorskie
Lubelskie,1921,1507,414,78.448725,21.551275,Lubelskie
Łódzkie,911,645,266,70.801317,29.198683,Łódzkie


In [26]:
plot_df3(df,basiw.image_dir, 'zgony_wspol_procent_od2021-07-14.jpg')

Sprawdzenie:

In [27]:
# print('brak współistniejących: ', dfd4.query('(Województwo == "Podlaskie") & (czy_wspolistniejace == "N")').shape[0])
# print('współistniejące: ', dfd4.query('(Województwo == "Podlaskie") & (czy_wspolistniejace == "T")').shape[0])

# display(dfd4.query('(Województwo == "Podlaskie") & (czy_wspolistniejace == "N")').sum())
# display(dfd4.query('(Województwo == "Podlaskie") & (czy_wspolistniejace == "T")').sum())

# display(dfd4.query('(Województwo == "Podlaskie") & (czy_wspolistniejace == "N")').groupby('Województwo').sum())
# display(dfd4.query('(Województwo == "Podlaskie") & (czy_wspolistniejace == "T")').groupby('Województwo').sum())

OK, są drobne różnice, bo liczba zaraportowanych zgonów w danej komórce nie zawsze jest równa 1.

W związku z powyższym rysunkiem, nie ma sensu robić rysunków poniżej, bo choroby towarzyszące są raportowane jak bądź.

In [28]:
pop_df

,mieszkańców w danej grupie wiekowej,kobiet w danej grupie wiekowej,mężczyzn w danej grupie wiekowej
Age group,,,
0-18,6909092.0,3362871.0,3546221.0
19-24,2391135.0,1169959.0,1221176.0
25-34,5378329.0,2637687.0,2740642.0
35-44,6101624.0,3008948.0,3092676.0
45-54,4789676.0,2395882.0,2393794.0
55-64,5104262.0,2668881.0,2435381.0
65-74,4195973.0,2362488.0,1833485.0
75-84,1912001.0,1218241.0,693760.0
85-94,756428.0,540453.0,215975.0


In [29]:
# dfpopwoj.to_dict()['Ludność']
pop_woj_df

,mieszkańców w województwie
Województwo,
Dolnośląskie,2884011
Kujawsko-Pomorskie,2054163
Lubelskie,2086403
Lubuskie,1003150
Łódzkie,2426806
Małopolskie,3408505
Mazowieckie,5419963
Opolskie,973298
Podkarpackie,2115578


In [30]:
from_date = datetime.date(2021,7,14)

df_wspol_woj = prepare_df2(
    deaths_df = basiw.dfd.query('czy_wspolistniejace == "T"'), 
    x_col ='Województwo', 
    x_col_name = 'Województwo',
    denominator_df=pop_woj_df,
    denominator_col='mieszkańców w województwie', # Ma być: 'mieszkańców w województwie'
    from_date=from_date
    )

# df_wspol_woj

In [31]:
x_col = 'Województwo'
title_str = "z chorobami współistniejącymi" 
filename_str = "woj_ogolem_z_wspol"
denominator_col = "mieszkańców w województwie"
title = "Liczba zgonów na COVID-19 u " + denominator_col + " " + title_str + "<br>na 100 tys. "+ denominator_col +\
        "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych (od " + str(from_date) + ")"
df_to_plot = df_wspol_woj.sort_values(['Zgony nie w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col], ascending=False)
plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title=title , 
    filename_str=filename_str,  
    from_date = from_date, 
    df = df_to_plot, 
    image_dir = basiw.image_dir,
    legend_y=-0.5
    )

In [32]:
from_date = datetime.date(2021,7,14)

title_str = "bez chorób współistniejących" 
filename_str = "woj_ogolem_bez_wspol"

x_col = 'Województwo'
denominator_col = "mieszkańców w województwie"

df_bez_wspol_woj = prepare_df2(
    deaths_df = basiw.dfd.query('czy_wspolistniejace == "N"'), 
    x_col =x_col, 
    x_col_name = x_col,
    denominator_df=pop_woj_df,
    denominator_col=denominator_col, 
    from_date=from_date
    )


title = "Liczba zgonów na COVID-19 u " + denominator_col + " " + title_str + "<br>na 100 tys. "+ denominator_col +\
        "<br>w podziale na w pełni zaszczepionych i nie w pełni zaszczepionych (od " + str(from_date) + ")"
df_to_plot = df_wspol_woj.sort_values(['Zgony nie w pełni zaszczepionych ' + denominator_col + ' na 100 tys. ' + denominator_col], ascending=False)
plot_df2(
    x_col=x_col, 
    denominator_col=denominator_col, 
    title=title , 
    filename_str=filename_str,  
    from_date = from_date, 
    df = df_to_plot, 
    image_dir = basiw.image_dir,
    legend_y=-0.5
    )